In [1]:
from cluster.cluster import make_clusters
from input.grid import grid as Grid
from graph.make_graph import make_graph
from graph.find_path import find_optimal_path
from random_state_generator.random_state_generator import random_state_generator
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
def run_experiment(length,height,num):
    
    k = 2 #how much we want to shorten the grid into; for example for k=2 100x100 turns into 50x50; for k=5 100x100 turns into 20x20
    rectangle,source,dest = random_state_generator(length,height,num,gap=k+3)
    grid = Grid(rectangle)

    start = time.time()
    clusters = make_clusters(rectangle)
    graph = make_graph(clusters,source,dest)
    grid.add_graph(graph)

    # optimal path

    opt_path,opt_path_len = find_optimal_path(graph,source,dest)
    end = time.time()
    time_taken = end-start
    # print(f"time taken :{end-start}, path length: {opt_path_len}")


    # meta


    grid.make_meta_grid(k=k)
    start = time.time()
    meta_clusters = make_clusters(grid.meta_rect)
    meta_src = (grid.meta_rect.shape[0]-1,0)
    meta_dest = (0,grid.meta_rect.shape[1]-1)
    meta_graph = make_graph(meta_clusters,meta_src,meta_dest)
    grid.add_graph(meta_graph,type="meta")


    # reverse mapping
    met_path_graph = grid.print_reverse_meta_grid(meta_graph,k,print_grid=False)
    src = (meta_src[0]*k,meta_src[1]*k)
    dst = (meta_dest[0]*k,meta_dest[1]*k)
    meta_opt_path,meta_opt_path_len = find_optimal_path(met_path_graph,src,dst)
    end = time.time()
    time_taken_rev = end - start
    # print(f"time taken :{end-start}, path length: {meta_opt_path_len}")
    
    return opt_path_len,meta_opt_path_len,time_taken,time_taken_rev

param_list = [[40,30,3],[50,60,6],[400,500,7],[500,4200,15],
              [600,50,3],[1000,80,6],[500,500,8],[5000,5000,25],
              [4000,2000,13],[5000,1500,6]
             ]

arr = np.array(param_list)
opl_list = []
mopl_list = []

inc_len_list = []

tt_list = []
ttr_list = []

decr_t_list = []



for l,h,n in tqdm(param_list):
    a,b,c,d = run_experiment(l,h,n)
    opl_list.append(a)
    mopl_list.append(b)
    tt_list.append(c)
    ttr_list.append(d)
    inc_l,dec_t = None,None
    if a>0:
        inc_l = ((b-a)/a)*100
    if c>0:
        dec_t = ((c-d)/c)*100
    inc_len_list.append(inc_l)
    decr_t_list.append(dec_t)
    
arr = np.concatenate((arr,np.array([opl_list]).T),axis=1)
arr = np.concatenate((arr,np.array([mopl_list]).T),axis=1)
arr = np.concatenate((arr,np.array([tt_list]).T),axis=1)
arr = np.concatenate((arr,np.array([ttr_list]).T),axis=1)
arr = np.concatenate((arr,np.array([inc_len_list]).T),axis=1)
arr = np.concatenate((arr,np.array([decr_t_list]).T),axis=1)

# print(arr)

column_names = ["l","h","n","path_len","path_len_1","time","time_1","%l inc","%t decr"]
df = pd.DataFrame(arr,columns=column_names)
df

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:55<00:00, 23.58s/it]


,l,h,n,path_len,path_len_1,time,time_1,%l inc,%t decr
0,40.0,30.0,3.0,49.232651,48.534502,0.0,0.0,-1.418062,None
1,50.0,60.0,6.0,83.750097,85.537659,0.0,0.0,2.134399,None
2,400.0,500.0,7.0,663.057191,662.743447,0.063716,0.016389,-0.047318,74.277343
3,500.0,4200.0,15.0,4249.719965,4251.791441,1.091386,0.618583,0.048744,43.32131
4,600.0,50.0,3.0,601.140236,600.182104,0.030769,0.011078,-0.159386,63.99625
5,1000.0,80.0,6.0,1007.586386,1006.788646,0.082932,0.014777,-0.079173,82.181668
6,500.0,500.0,8.0,742.795802,743.100894,0.244324,0.053986,0.041073,77.903996
7,5000.0,5000.0,25.0,8644.013504,8648.799566,23.821283,5.351525,0.055369,77.53469
8,4000.0,2000.0,13.0,4976.925123,4979.212164,6.961655,1.867003,0.045953,73.181623
9,5000.0,1500.0,6.0,5505.070966,5506.139042,5.399854,1.619316,0.019402,70.011858
